In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [3]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
#CLEAN TRAINING DATA
bounds = [0.0, 0.5]
tx_clean, ind_remov = treat_undefined_values(bounds, tX)
ind_remov

[4, 5, 6, 12, 26, 27, 28]

In [5]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)
tx_augmented = build_poly_all_features(tx,2)
y,tx = build_model_data(tx_augmented,y)
y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)


In [6]:
num_samples, num_features
tx.shape

(250000, 47)

In [7]:
from compute_gradient import *
from cost import *
from implementations import *

In [10]:
#Initial weight


#Maximum iterations through the whole set
max_iter = 100

#0.001 nan
#0.0005 nan

#Step-size
gammas = np.logspace(-6,-3, 7)
lambdas = np.logspace (-5,0,7)


#Regularization factor
# lambda_ = 0.01 142000 loss 10 000 iters
#lambda_ = 0.001 141000 loss 10 000 iters
#lambda_ = 0.02
#loss_min = np.inf
tx.shape,y.shape,y

((250000, 47), (250000, 1), array([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]))

In [ ]:
losses_tr = np.zeros((len(gammas),len(lambdas)))
losses_val = np.zeros((len(gammas),len(lambdas)))

k_fold = 2
seed = 1
loss_min = np.inf
initial_w = np.zeros((tx.shape[1],1))

#CHOOSE LAMBDA AND GAMMA
for ind_lambda, lambda_ in enumerate(lambdas):
    for ind_gamma ,gamma_ in enumerate (gammas):
        loss_training = 0
        loss_validation = 0
        for i,k in enumerate(range(k_fold)):
            
            tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
            w, loss_train_k = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iter, gamma_)
            loss_val_k = reg_logistic_loss (y_val,tx_val,w,lambda_)/len(y_val)
            loss_train_k /= len(y_train)
            loss_training += loss_train_k
            loss_validation += loss_val_k
        
        
        loss_training /= k_fold
        loss_validation /= k_fold
        print(loss_training)
        print(loss_validation)
        if (loss_validation < loss_min):
            weights_star = w
            lambda_star = lambda_
            gamma_star = gamma_
            loss_min = loss_validation
        
        losses_tr[ind_lambda][ind_gamma] = loss_training
        losses_val[ind_lambda][ind_gamma] = loss_validation



0.693038900853815
0.693038626049674
0.6926698493894505
0.6926687692757123
0.691516208195458
0.6915115249552872
0.6879823551204608
0.6879650486461031
0.6778106799235489
0.677749738877403
0.6531265553918006
0.6529375662217655
0.613318196161167
0.6126913813140342
0.6082108341655306
0.6082106929122877
0.6081276245536795
0.6081271735898788
0.6078662653520552
0.6078647975095669
0.6070571411113745
0.6070520866097321
0.6046614904898112
0.6046415394000191
0.5984393561679111
0.5983416034460913
inf
0.5866080968985525
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf


In [ ]:
losses_tr, losses_val

In [179]:
#PREDICT TRAINING ACCURACY
y_pred_training = predict_labels(weights_star, tx,'logistic')
count = 0
for i in range(num_samples):
    if (y_pred_training[i] == -1 and y[i] == 0):
        count += 1
    elif (y_pred_training[i] == y[i]):
        count += 1
accuracy = (count *100) / num_samples

accuracy

65.7348

In [10]:
#HYPER TUNING OF PARAMETERS

losses_tr = np.zeros((len(gammas),len(lambdas)))
losses_val = np.zeros((len(gammas),len(lambdas)))

k_fold = 2
seed = 1

for ind_lambda, lambda_ in enumerate(lambdas):
    for ind_gamma ,gamma_ in enumerate (gammas):
        for ind_degree, degree in enumerate (degrees):
            tx = build_poly_all_features(tx,degree)
            for i,k in enumerate(range(k_fold)):
                
                tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
                w, loss_training = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iter, gamma)
                loss_val = reg_logistic_loss (y_val,tx_val,w,lambda_)/len(y_val)
                loss_training /= len(y_train)
                losses_tr[ind_lambda][ind_gamma]= loss_training
                losses_val [ind_lambda][ind_gamma] = loss_val
            
            
                if loss_training < loss_min:
                    weights_star = w
                    gamma_star = gamma
                    lambda_star = lambda_
                    loss_min = loss_training

109756.20152532344
106358.24427164088
103998.8971821258
101962.78875222043
100254.79069723404
99122.38969705586
98024.67377478634
96873.45443026749
96200.26310447392
95512.88918771365
94633.74422621673
94174.9225446923
93700.44105454849
92984.60449754844
92635.08271871509
92283.37906065678
91681.76554292584
91390.07583801802
91114.97847155234
90602.76483372312
90341.48166176202
90117.24236954766
89680.53116548403
89434.13306026228
89245.18785906598
88875.1919369561
88634.3119818995
88470.7695883172
88161.42409864564
87920.26124928558
87775.4829514335
87522.16153524394
87285.51012460973
87146.38468264748
86945.38989317586


/Users/Sofia/Desktop/ML_Project1/cost.py:20: RuntimeWarning: divide by zero encountered in log
  loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))


inf
86587.41363136146
86422.20268752072
nan
nan
85945.01798911016
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
84740.28664937796
nan
nan
87838.05540500928
nan
nan
90451.7818387172
nan
91773.01016959178
92466.24412926171
93031.61730846028
93469.53342641865
93943.10752045868
94390.89359130901
94689.38939216967
94996.1247354117
95353.57864514536
95550.10579544837
95735.97691935896
96027.5590714678


In [11]:
losses_tr

array([[0.62399588, 0.59473672, 0.57720389, 0.5650518 , 0.55581272,
        0.54834265, 0.54205106],
       [0.53660694, 0.531808  , 0.52752368, 0.52371516,        inf,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan, 0.55819194, 0.56634763,
        0.57212376, 0.57616766]])

In [12]:
losses_val

array([[0.62358287, 0.59419693, 0.57664055, 0.56451943, 0.55534669,
        0.54796646, 0.54177972],
       [0.53644946, 0.53176947, 0.52760732, 0.52387337, 0.52050458,
        0.5174515 , 0.51467054],
       [0.51213306, 0.50980579, 0.50766861, 0.50570216, 0.50388547,
        0.50219951, 0.50062936],
       [0.49920404, 0.49786857, 0.49661534, 0.4954376 , 0.49432922,
        0.49328466, 0.49229889],
       [0.49162306, 0.49097199, 0.49034576, 0.48974414, 0.48916671,
        0.48861291, 0.48808202],
       [0.48935751, 0.49060756, 0.49181166, 0.49296242, 0.49405869,
        0.49510162, 0.49609291],
       [0.5151914 , 0.53307717, 0.54757207, 0.55853499, 0.56649348,
        0.57213713, 0.57609077]])

In [198]:
#LOAD TEST SET
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [203]:
#CLEAN AND STANDARDIZE TEST SET
tX_test_clean = np.delete(tX_test, ind_remov, axis=1)
tX_test_clean.shape


(568238, 23)

In [206]:
tx_test_clean = remove_undefined_values (tX_test_clean)
tx_test_clean.shape

(568238, 23)

In [207]:
tx_test_clean, _, _ = standardize(tx_test_clean,0)


In [210]:
tx_test_augmented = build_poly_all_features(tx_test_clean,2)

(568238, 23)

In [213]:
tx_test_augmented.shape

(568238, 46)

In [214]:
#BUILD TEST MODEL
ytest, tx_test = build_model_data(tx_test_augmented,ytest)

In [215]:
#PREDICT LABELS
OUTPUT_PATH = 'data/submission_reg_logistic-degre2.csv'
y_pred = predict_labels(weights_star, tx_test,'logistic')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [31]:
lambda_star, gamma_star,loss_min

(1.0, 5e-05, inf)

In [23]:
gammas

array([1.00000000e-06, 3.16227766e-06, 1.00000000e-05, 3.16227766e-05,
       1.00000000e-04, 3.16227766e-04, 1.00000000e-03])

In [24]:
lambdas

array([1.00000000e-05, 6.81292069e-05, 4.64158883e-04, 3.16227766e-03,
       2.15443469e-02, 1.46779927e-01, 1.00000000e+00])